# Goodreads reviews of Romance books

In [1]:
import pandas as pd
import os
from os.path import join
import stanza
import re
from nltk import sent_tokenize
from collections import Counter
from langdetect import detect
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
ana = SentimentIntensityAnalyzer()
import requests
from langdetect import detect
import nltk
from bs4 import BeautifulSoup
from nltk import word_tokenize,pos_tag
import matplotlib.pyplot as plt
import seaborn as sns
import random
from bs4.element import Comment
import credentials
key = credentials.key

import string
from nltk.corpus import stopwords
stopword_list = stopwords.words('english')
for mark in string.punctuation + '“”’—‘':
    stopword_list.append(mark)

def remove_punctuation(words):
    new_list= []
    for w in words:
        if w.isalnum():
            new_list.append( w )
    return new_list

def sorted_by_value( dict , ascending = True ):
    if ascending: 
        return {k: v for k, v in sorted(dict.items(), key=lambda item: item[1])}
    else:
        return {k: v for k, v in reversed( sorted(dict.items(), key=lambda item: item[1]))}
    
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'
]

def select_agent():
    n=random.randint(0,len(user_agents)-1)
    return user_agents[n]

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True



This notebook analyses reviews of books which were on the New York Times bestsellers lists in 2015. 

In [2]:
directory = 'Romance'
if not os.path.exists(directory):
    os.makedirs(directory)

In [3]:
df = pd.read_csv('romance_goodreads.csv')
print(f'The CSV file contains {df.shape[0]} records.')

The CSV file contains 239 records.


In [7]:

for url in df['Antwort_Link']:
    gr_id = re.sub(r'https:\/\/www[.]goodreads[.]com\/book\/show/','',url)
    gr_id = gr_id[ :gr_id.index('-') ]
    path = join(directory,f'reviews_{gr_id}.txt')
    if not os.path.exists(path):
        out = open(path,'w',encoding='utf-8')
        api_call = 'https://www.goodreads.com/book/show.json'
        api_call += f'?key={key}&id={gr_id}&text_only=true'
        headers = {'User-Agent': select_agent() }
        response = requests.get( api_call,headers=headers)
        json_data = response.json()
        reviews_widget = json_data['reviews_widget']

        soup = BeautifulSoup( reviews_widget ,"lxml")

        iframe = soup.find("iframe")

        pages_left = True

        url = iframe.get("src")

        i = 1
        while pages_left == True:
            if re.search( r'[&]page',url,re.IGNORECASE):
                url = url[:url.index('&page')]
            url += '&page=' + str(i)
            print(f" {url}")

            headers = {'User-Agent': select_agent() }
            response_reviews = requests.get(url,headers=headers)
            html_page = response_reviews.text
            if re.search(r'No\s+reviews\sfound',html_page,re.IGNORECASE):
                pages_left = False
            elif i == 100:
                pages_left = False


            soup = BeautifulSoup( html_page ,"lxml")

            review_links = soup.find_all('a',{'class':'gr_more_link'})
            for link in review_links:
                url_review = link.get('href')
                print(url_review)
                headers = {'User-Agent': select_agent() }
                response_review = requests.get(url_review,headers=headers)
                html_page2 = response_review.text
                soup = BeautifulSoup( html_page2 ,"lxml")

                review_text = soup.find('div',{'class':'reviewText'})
                full_text = ''
                if review_text is not None:
                    full_text = review_text.text 
                    full_text = re.sub('...more','',full_text)

                language = 'Unknown'
                try:
                    language = detect(full_text)
                except:
                    print('Lanuage could not be detected')

                if language =='en':
                    out.write(full_text.strip()+'\n')


            i+=1

    out.close()

 https://www.goodreads.com/api/reviews_widget_iframe?did=DEVELOPER_ID&format=html&isbn=B0BY9DRDK3&links=660&review_back=fff&stars=000&text=000&page=1
https://www.goodreads.com/review/show/5493413035?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/6028758408?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5427534481?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5985686450?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/6383529197?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5320443621?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5446310546?utm_campaign=reviews&utm_medium=widget
 https://www.goodreads.com/api/reviews_widget_iframe?did=DEVELOPER_ID&format=html&isbn=B0BY9DRDK3&links=660&review_back=fff&stars=000&text=000&page=2
https://www.goodreads.com/review/show/6334543621?utm_campaign=reviews&utm_medium=wid

https://www.goodreads.com/review/show/6288711780?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5758686578?utm_campaign=reviews&utm_medium=widget
 https://www.goodreads.com/api/reviews_widget_iframe?did=DEVELOPER_ID&format=html&isbn=B0BY9DRDK3&links=660&review_back=fff&stars=000&text=000&page=11
 https://www.goodreads.com/api/reviews_widget_iframe?did=DEVELOPER_ID&format=html&isbn=1538767651&links=660&review_back=fff&stars=000&text=000&page=1
https://www.goodreads.com/review/show/5457672209?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5169316472?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5407114626?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/6387379245?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5510865364?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5486709449?utm_campaign=reviews&utm_medium=wi

https://www.goodreads.com/review/show/6430566414?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5555632913?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5450136023?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5838555351?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5480582813?utm_campaign=reviews&utm_medium=widget
Lanuage could not be detected
https://www.goodreads.com/review/show/5577311583?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5567967946?utm_campaign=reviews&utm_medium=widget
 https://www.goodreads.com/api/reviews_widget_iframe?did=DEVELOPER_ID&format=html&isbn=1538767651&links=660&review_back=fff&stars=000&text=000&page=11
 https://www.goodreads.com/api/reviews_widget_iframe?did=DEVELOPER_ID&format=html&isbn=1990105297&links=660&review_back=fff&stars=000&text=000&page=1
https://www.goodreads.com/review/show/5105973586?utm_

https://www.goodreads.com/review/show/5895166331?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5801567564?utm_campaign=reviews&utm_medium=widget
 https://www.goodreads.com/api/reviews_widget_iframe?did=DEVELOPER_ID&format=html&isbn=1990105297&links=660&review_back=fff&stars=000&text=000&page=10
https://www.goodreads.com/review/show/5320903168?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5411910365?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5915189851?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5801356928?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5114736367?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5817799082?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5090843581?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5750126947?utm_camp

Lanuage could not be detected
https://www.goodreads.com/review/show/6331942270?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5420379203?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5914288560?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/6338526137?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/6467128810?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5692878460?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/6043184077?utm_campaign=reviews&utm_medium=widget
 https://www.goodreads.com/api/reviews_widget_iframe?did=DEVELOPER_ID&format=html&isbn=B0BWX1LY33&links=660&review_back=fff&stars=000&text=000&page=10
https://www.goodreads.com/review/show/5499217141?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5946585535?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/r

...

https://www.goodreads.com/review/show/5545271586?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5408989527?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5334851920?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5329637945?utm_campaign=reviews&utm_medium=widget
 https://www.goodreads.com/api/reviews_widget_iframe?did=DEVELOPER_ID&format=html&isbn=0593243390&links=660&review_back=fff&stars=000&text=000&page=5
https://www.goodreads.com/review/show/5081452333?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5369870610?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5379018562?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5293178470?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5598264638?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5481131109?utm_campa

https://www.goodreads.com/review/show/5572083558?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5143784596?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5584951944?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5330609401?utm_campaign=reviews&utm_medium=widget
 https://www.goodreads.com/api/reviews_widget_iframe?did=DEVELOPER_ID&format=html&isbn=1250826799&links=660&review_back=fff&stars=000&text=000&page=6
https://www.goodreads.com/review/show/5320310763?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5893525423?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5318924821?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/4027013787?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5460645993?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5485843872?utm_campa

https://www.goodreads.com/review/show/6212156460?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5330563829?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5340213521?utm_campaign=reviews&utm_medium=widget
 https://www.goodreads.com/api/reviews_widget_iframe?did=DEVELOPER_ID&format=html&isbn=0063054892&links=660&review_back=fff&stars=000&text=000&page=7
https://www.goodreads.com/review/show/5160988686?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5427569158?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5028011162?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5987789021?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5219177319?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5337480517?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5380677744?utm_campa

https://www.goodreads.com/review/show/4799473784?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/4983063480?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5111354284?utm_campaign=reviews&utm_medium=widget
 https://www.goodreads.com/api/reviews_widget_iframe?did=DEVELOPER_ID&format=html&isbn=125021744X&links=660&review_back=fff&stars=000&text=000&page=8
https://www.goodreads.com/review/show/5298020026?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5329592150?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5584682258?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/6131928903?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/5515372795?utm_campaign=reviews&utm_medium=widget
https://www.goodreads.com/review/show/3078514865?utm_campaign=reviews&utm_medium=widget
 https://www.goodreads.com/api/reviews_widget_iframe?did=D

KeyboardInterrupt: 

In [8]:
reviews =os.listdir(directory)
reviews = [join(directory,file) for file in reviews if re.search(r'txt$',file)]

Reviews in langauges other than English are left out of consideration. 

## Part of speech tagging

What are the most common adjectives in the reviews? 

In [9]:
pos_data = []
stopwords = ['i','not','so']
adj_adv = []

freq = Counter()


for review in reviews:
    
    nr_modal = 0
    nr_comparative = 0
    vbp = 0
    vbz = 0

    full_text = ''
    with open(review) as fh:
        adj_adv = []
        full_text = fh.read().lower()

        words = word_tokenize(full_text)
        words = remove_punctuation(words)
        words = [word for word in words if word not in stopwords]
        pos = pos_tag(words)
        

        for p in pos:
            if re.search(r'^JJ',p[1]) or re.search(r'^RB',p[1]):
                if p[0] not in stopword_list:
                    adj_adv.append(p[0])
                
        freq.update(adj_adv)
        
for word,count in freq.most_common(30):
    print(word,count)

really 2619
much 2327
also 2314
even 1872
first 1777
good 1510
never 1245
many 1187
still 1181
well 1111
little 1100
always 952
new 919
best 805
next 762
great 747
long 744
main 730
better 718
ever 716
different 699
together 694
last 682
definitely 677
back 658
actually 652
absolutely 630
favorite 600
enough 580
right 574


## Sentiment Analysis

What are the most negative and the most positive sentences?

In [10]:
data = []
nr_sentences = 0
count_positive = 0
count_negative = 0
romance_sentences = dict()


for file in reviews: 

    text = open(file,encoding='utf-8')
    full_text = text.read()
    sentences = sent_tokenize(full_text)
    for sentence in sentences:
        scores = ana.polarity_scores(sentence)
        nr_sentences += 1
        romance_sentences[sentence] = scores['compound']
        if scores['compound'] > 0.6:
            count_positive+=1
        elif scores['compound'] < -0.6:
            count_negative+=1


count_neutral = nr_sentences - count_positive-count_negative

data.append( ['Romance',count_neutral/nr_sentences,count_positive/nr_sentences,count_negative/nr_sentences])
df = pd.DataFrame(data,columns=['reviews','neutral','positive','negative'])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


y_axis = 'reviews'

fig = plt.figure( figsize=( 8 , 3 ) )

df['combined'] = df['positive'] + df['neutral'] + df['negative']
graph = sns.barplot( data=df , x='combined', y=y_axis , label = 'Negative', color = '#c90e0e' )
df['combined'] = df['combined'] - df['negative']
graph = sns.barplot( data=df , x='combined', y=y_axis , color = '#ebdfdf')
df['combined'] = df['combined'] - df['neutral']
graph = sns.barplot( data=df , x='combined', y=y_axis , label = 'Positive' ,color = '#1b66de')

graph.set_title('Sentiment analysis' , size = 20) 
graph.set_xlabel('Scores' , size = 14) 
graph.set_ylabel('Palestine' , size = 14 )

plt.xticks(rotation= 90)

# The next line places the legend outside out the plot
plt.legend( bbox_to_anchor=(1.05, 1),loc=2, borderaxespad=0.);

plt.show()

What are the 10 most negative sentences?

In [11]:
iterator = iter(sorted_by_value(romance_sentences).items())
for i in range(10):
    item = next(iterator)
    print(item[0], item[1])

i saw the enemies to lovers arc coming from a fucking mile away but once a nikmai shipper, always a nikmai shipper.jesus christ it’s been a while, so here are the characters:- wulf (armul) the good immune system - glorian the badass bitch- tunuva the also bad bitch- dumai the god like damn- nikeya the reformed tory- esbar the i sort of liked her - canthe the bitch- unora the emperor fucker- guma the weird old man- siyu the repenter- fyredel hates inys - valeysa is mentioned once- thrit the gay ✌️- karl the absolute fucking dickhead- bardholt got downed by the sea 😭😭😭😭- sabran the bad bitch- florell the women’s rights activist - robart the cultist- imin was very nice - ninura the good girl - jeda growls a lot- furtia 😭😭😭yeah this book was a rollercoaster and i may have cried a couple of times.whatever samantha puts in these fucking books i ne of it, because my fucking god if i don’t get official sabread i will riot in the streets.did dumai die? -0.9974
<3trigger + content warnings: loss

What are the 10 most positive sentences?

In [12]:
iterator = iter(sorted_by_value(romance_sentences,ascending=False).items())
for i in range(10):
    item = next(iterator)
    print(item[0], item[1])

Glorian is the daughter of two monarchs, but happiest in their shadow.Dumai is a godsinger, who has spent her life trying to wake the dragons from their eternal slumber.But when Dreadmount erupts, bringing with it an age of terror and violence, these women must find the strength to protect humankind...I loved Priory, but I adored this.I don’t know why, but I just felt so mu connected to characters, and loved them so much!Characters tend to be what makes or breaks a book for me, and these characters definitely made it!Glorian, Tunuva, Dumai, Esbar, and Nikaya, this cast of women were everything to me.What I particularly love about this series, this world, and Samantha Shannon’s writing, is how she writes female relationships.Not just the beautiful sapphic relationships, but also the grandmothers, the mothers, the sisters, adopted or birth, they were an absolutely stunning representation of how women can protect and help each other.Glorian was my favourite character, and I was constantly